## 实验目标: 测试不同数据集的workload对于price模型的贡献度

方案: 保持训练集的sql总数不变(每个数据集对应5W条sql用于训练)，但被用于测试的数据集所对应的sql中，选用 \mathfs{A}% 的sql用于训练，剩下的 1 - \mathfs{A}% 的sql用重复的sql代替。设 min(\mathfs{A}%) = 50% 
    四个测试数据集只有imdb的连接方式是 Star，其他三个的连接方式都是 Mixed。

原始模型：model_params.pth \
    结果：\
    imdb loss: 1.6293717622756958 \
    imdb q-error: 30%: 1.3949   50%: 1.7771   80%: 4.0716   90%: 8.3952   95%: 15.4516   99%: 70.8889 \
    stats loss: 2.834393262863159 \
    stats q-error: 30%: 1.3827   50%: 1.8709   80%: 5.4934   90%: 12.4606   95%: 35.5513   99%: 579.6716 \
    ergastf1 loss: 1.3214871883392334 \
    ergastf1 q-error: 30%: 1.2311   50%: 1.4315   80%: 3.1244   90%: 6.4412   95%: 14.3196   99%: 67.9744 \
    genome loss: 1.5354844331741333 \
    genome q-error: 30%: 1.3125   50%: 1.6545   80%: 3.5951   90%: 5.1724   95%: 15.6697   99%: 114.4227

控制变量数据集：Baseball 原因：表的数量最多，属性的数量最多，连接关系的数量第二多，连接方式是 Mixed。 

测试模型：model_params_but_50%_baseball_repeated.pth ---- baseball训练集中50%的sql是重复的 \
    结果： \
    imdb loss: 2.4607784748077393 \
    imdb q-error: 30%: 1.4052   50%: 1.9268   80%: 5.3653   90%: 12.2204   95%: 36.8323   99%: 351.8588 \
    stats loss: 4.839287757873535 \
    stats q-error: 30%: 1.4067   50%: 2.2407   80%: 9.7414   90%: 33.6225   95%: 115.5289   99%: 3570.0361 \
    ergastf1 loss: 2.3229031562805176 \
    ergastf1 q-error: 30%: 1.2658   50%: 1.6869   80%: 5.767   90%: 12.1195   95%: 35.1984   99%: 168.052 \
    genome loss: 3.423879861831665 \
    genome q-error: 30%: 1.3384   50%: 1.9047   80%: 10.7782   90%: 19.3522   95%: 52.6266   99%: 376.3436

测试模型： \
    结果：model_params_but_50%_baseball_used.pth ---- 只使用baseball训练集中50%的sql \
    imdb loss: 1.5814220905303955 \
    imdb q-error: 30%: 1.3531   50%: 1.6338   80%: 3.9253   90%: 6.6827   95%: 16.8235   99%: 78.0464 \
    stats loss: 5.702045440673828 \
    stats q-error: 30%: 1.5224   50%: 2.4428   80%: 13.1791   90%: 43.5221   95%: 143.4585   99%: 5914.5057 \
    ergastf1 loss: 2.3291282653808594 \
    ergastf1 q-error: 30%: 1.2663   50%: 1.7088   80%: 5.1405   90%: 13.4376   95%: 26.6408   99%: 262.1947 \
    genome loss: 2.91455340385437 \
    genome q-error: 30%: 1.217   50%: 1.6557   80%: 4.6641   90%: 25.4964   95%: 63.3777   99%: 376.9749 
    


#### 初步分析1：
    训练集重复时，测试结果显示性能都出现了下降。训练集不重复时，只有IMDB数据集上的测试结果没有出现明显性能下降。 
    猜想：应该可能的原因是baseball数据集的连接方式和imdb不同因此没有影响，还应测试改变连接方式是star的数据集的训练用sql数量。 


控制变量数据集：Airline 原因：连接方式是 Star，在这种连接方式中表的数量最多，属性的数量最多，连接关系的数量最多。

测试模型： \
    结果：model_params_but_50%_airline_used.pth ---- 只使用airline训练集中50%的sql \
    imdb loss: 1.6031652688980103 \
    imdb q-error: 30%: 1.3328   50%: 1.6361   80%: 4.043   90%: 6.9649   95%: 14.9615   99%: 92.8439 \
    stats loss: 5.822782516479492 \
    stats q-error: 30%: 1.5002   50%: 2.418   80%: 13.6211   90%: 51.609   95%: 168.8235   99%: 6451.1508 \
    ergastf1 loss: 1.9035067558288574 \
    ergastf1 q-error: 30%: 1.264   50%: 1.6402   80%: 4.0963   90%: 9.4962   95%: 23.6397   99%: 148.0107 \
    genome loss: 1.0353407859802246 \
    genome q-error: 30%: 1.0831   50%: 1.2122   80%: 1.8747   90%: 5.3969   95%: 13.9338   99%: 62.2175

#### 初步分析2：
    imdb: 
        原始模型 VS model_params_but_50%_baseball_used.pth：90%: 8.3952   95%: 15.4516   99%: 70.8889 -> 90%: 6.6827   95%: 16.8235   99%: 78.0464 
        原始模型 VS model_params_but_50%_airline_used.pth： 90%: 8.3952   95%: 15.4516   99%: 70.8889 -> 90%: 6.9649   95%: 14.9615   99%: 92.8439 
        model_params_but_50%_airline_used 更差 
    stats： 
        原始模型 VS model_params_but_50%_baseball_used.pth：90%: 12.4606   95%: 35.5513   99%: 579.6716 -> 90%: 43.5221   95%: 143.4585   99%: 5914.5057 
        原始模型 VS model_params_but_50%_airline_used.pth： 90%: 12.4606   95%: 35.5513   99%: 579.6716 -> 90%: 51.609   95%: 168.8235   99%: 6451.1508 
        model_params_but_50%_airline_used 更差 
    ergastf1: 
        原始模型 VS model_params_but_50%_baseball_used.pth：90%: 6.4412   95%: 14.3196   99%: 67.9744 -> 90%: 13.4376   95%: 26.6408   99%: 262.1947 
        原始模型 VS model_params_but_50%_airline_used.pth： 90%: 6.4412   95%: 14.3196   99%: 67.9744 -> 90%: 9.4962   95%: 23.6397   99%: 148.0107 
        model_params_but_50%_baseball_used 更差 
    genome: 
        原始模型 VS model_params_but_50%_baseball_used.pth：90%: 5.1724   95%: 15.6697   99%: 114.4227 -> 90%: 19.3522   95%: 52.6266   99%: 376.3436 
        原始模型 VS model_params_but_50%_airline_used.pth： 90%: 5.1724   95%: 15.6697   99%: 114.4227 -> 90%: 5.3969   95%: 13.9338   99%: 62.2175 
        model_params_but_50%_baseball_used 更差 

    不考虑 stats 数据集的话，还是哪个连接类型的sql减少了，哪个连接类型的模型性能会受影响更大。大概可以证实 初步分析1 的猜想。 
    猜想：同一连接方式的不同训练集中，越复杂的训练集起到的作用越大。

控制变量数据集：SSB 原因：连接方式是 Star，在所有训练集中 Star 连接方式的训练集数量不多，训练集的变化导致的性能变化可能更加明显。SSB训练集的结构是同类连接中最简单的。 

测试模型： \
    结果：model_params_but_50%_ssb_used.pth ---- 只使用ssb训练集中50%的sql \
    imdb loss: 1.6757985353469849 \
    imdb q-error: 30%: 1.3434   50%: 1.7224   80%: 3.9324   90%: 7.2447   95%: 15.3511   99%: 148.2468 \
    stats loss: 5.5983452796936035 \
    stats q-error: 30%: 1.4971   50%: 2.4807   80%: 12.9348   90%: 45.185   95%: 148.279   99%: 4191.3165 \
    ergastf1 loss: 2.5355474948883057 \
    ergastf1 q-error: 30%: 1.2721   50%: 1.6989   80%: 5.2201   90%: 11.6417   95%: 30.5691   99%: 349.9879 \
    genome loss: 1.458478569984436 \
    genome q-error: 30%: 1.3792   50%: 1.7662   80%: 4.8004   90%: 6.9155   95%: 9.3291   99%: 60.233

#### 初步分析3：
    和 model_params_but_50%_airline_used.pth 相比：
        imdb: 
            model_params_but_50%_airline_used.pth: 90%: 6.9649   95%: 14.9615   99%: 92.8439 
            model_params_but_50%_ssb_used.pth:     90%: 7.2447   95%: 15.3511   99%: 148.2468 
    反而是 ssb 训练集较小的效果更差，证伪了 初步分析2 的猜想。 ssb 数据集虽然是人工数据集，但相比于 airline 数据集更能提升模型性能。除了数据集的特征外，还应该考虑 workload 的影响。 airline 数据集虽然复杂的，但它的 workload 特征和 ssb 的 workload 特征相比并没有明显的差别（连接表的数量：2 - 7 vs 2 - 5），但 airline 数据集本身的表更多，可能导致对应训练集提供的“信息不足”？ 
    猜想：workload 的特征与数据集越匹配，越能对模型的性能有好的影响。

控制变量数据集：SAP 原因：数据集整体特征和 workload 的特征与 ssb 数据集相似，两者都很匹配，主要差异是 ssb 的表数比连接数大，而 SAP 的小，而这点反而和 airline 数据集的特点是一样的。预期结果是50% sap 训练集的对应模型的性能应和50% ssb 训练集的对应模型性能类似。 

测试模型： \
    结果：model_params_but_50%_sap_used.pth ---- 只使用sap训练集中50%的sql  \
    imdb loss: 1.4642819166183472 \
    imdb q-error: 30%: 1.331   50%: 1.6047   80%: 3.9869   90%: 6.3709   95%: 12.3625   99%: 73.6489 \
    stats loss: 5.800178050994873 \
    stats q-error: 30%: 1.489   50%: 2.4494   80%: 11.6606   90%: 44.3256   95%: 218.5632   99%: 5971.9011 \
    ergastf1 loss: 1.9272773265838623 \
    ergastf1 q-error: 30%: 1.2819   50%: 1.7353   80%: 4.716   90%: 11.6056   95%: 21.3093   99%: 78.2852 \
    genome loss: 1.3027501106262207 \
    genome q-error: 30%: 1.4086   50%: 1.7912   80%: 3.116   90%: 7.4964   95%: 9.7839   99%: 70.4891

#### 初步分析3：
    和原始模型，model_params_but_50%_ssb_used.pth，model_params_but_50%_airline_used.pth相比：   
    imdb：
        model_params.pth：                      90%: 8.3952   95%: 15.4516   99%: 70.8889
        model_params_but_50%_ssb_used.pth:      90%: 7.2447   95%: 15.3511   99%: 148.2468 
        model_params_but_50%_sap_used.pth：     90%: 6.3709   95%: 12.3625   99%: 73.6489
        model_params_but_50%_airline_used.pth： 90%: 6.9649   95%: 14.9615   99%: 92.8439 
    stats：
        model_params.pth：                      90%: 12.4606   95%: 35.5513   99%: 579.6716 
        model_params_but_50%_ssb_used.pth:      90%: 45.185   95%: 148.279   99%: 4191.3165 
        model_params_but_50%_sap_used.pth：     90%: 44.3256   95%: 218.5632   99%: 5971.9011
        model_params_but_50%_airline_used.pth： 90%: 51.609   95%: 168.8235   99%: 6451.1508
    ergastf1：
        model_params.pth：                      90%: 6.4412   95%: 14.3196   99%: 67.9744
        model_params_but_50%_ssb_used.pth:      90%: 11.6417   95%: 30.5691   99%: 349.9879
        model_params_but_50%_sap_used.pth：     90%: 11.6056   95%: 21.3093   99%: 78.2852
        model_params_but_50%_airline_used.pth： 90%: 9.4962   95%: 23.6397   99%: 148.0107
    genome：
        model_params.pth：                      90%: 5.1724   95%: 15.6697   99%: 114.4227
        model_params_but_50%_ssb_used.pth:      90%: 6.9155   95%: 9.3291   99%: 60.233
        model_params_but_50%_sap_used.pth：     90%: 7.4964   95%: 9.7839   99%: 70.4891  
        model_params_but_50%_airline_used.pth： 90%: 5.3969   95%: 13.9338   99%: 62.2175
    在 imdb，ergastf1 数据集上 model_params_but_50%_sap_used.pth 模型的性能接近原始模型，说明：（1）sap 数据集的作用很大，只需要50%就能起到足够的效果？or（2）sap 数据集的作用不大，去掉50%都不影响？
    在 stats 数据集上只有原始模型的性能最优，其他的都大幅度下降。
    在 ergastf1 数据集上原始模型的性能最优，但 model_params_but_50%_sap_used.pth 模型的性能接近原始模型，
    在 genome 数据集上，原始模型的性能最差，其他模型都比较接近。
    规律不明显，建议测试全部的 star 连接类型数据集。还剩 Carcinogenesis，Seznam，Telstra

    star 连接类型数据集以及workload类型分析：
    table数量 vs 连接关系的数量                  数据集     ------      
              >                                SSB，
              <                                Airline，Carcinogenesis，SAP，Seznam，Telstra，IMDB，
    表的数量 vs workload中最大连接表的数量       数据集      ------      表的数量 <= workload中连接表的数量 说明有机会学到完整的连接信息
            <=                                 Carcinogenesis，Seznam，SSB，Telstra，
            >                                  Airline，SAP，IMDB，
    
    测试模型： \
        结果：model_params_but_50%_carcinogenesis_used.pth ---- 只使用sap训练集中50%的sql  \
        imdb loss: 1.7479692697525024
        imdb q-error: 30%: 1.4093   50%: 1.8105   80%: 4.5625   90%: 7.8122   95%: 16.6908   99%: 135.1385
        stats loss: 5.543118476867676
        stats q-error: 30%: 1.4843   50%: 2.3265   80%: 11.1144   90%: 40.1652   95%: 170.4166   99%: 8056.6605
        ergastf1 loss: 1.5810344219207764
        ergastf1 q-error: 30%: 1.2519   50%: 1.5729   80%: 3.3572   90%: 8.5455   95%: 21.1032   99%: 97.5245
        genome loss: 2.1371867656707764
        genome q-error: 30%: 1.1839   50%: 2.0693   80%: 3.9452   90%: 14.2035   95%: 26.8704   99%: 160.1664

### 改进方法doremi

arxiv.org/abs/2305.10429

imdb pg's p-error 30% 50% 80% 90% 95% 99%: [1.0, 1.0, 1.2096, 1.7217, 2.1753, 14.5414] \
stats pg's p-error 30% 50% 80% 90% 95% 99%: [1.0, 1.001, 1.5453, 1.9138, 2.5732, 13.8785] \
ergastf1 pg's p-error 30% 50% 80% 90% 95% 99%: [1.0, 1.0, 1.2902, 1.5754, 1.9763, 6.3825] \
genome pg's p-error 30% 50% 80% 90% 95% 99%: [1.0256, 1.0392, 1.0405, 1.4158, 2.15, 4.6743] \

以price提供的预训练模型 model_params.pth 为 reference model，实现 doremi 方法后得到 main model 保存为 doremi_pretrain_params.pth ，其中包括 avg_domain_weights ，作为调优数据集构成的依据。 \
这组 avg_domain_weights 被保存为 avg_domain_weights_50000.json ，根据以这组权重采样训练数据(当前只是根据数量直接截断训练集)，重新训练 price 模型，保存为 doremi_params_50000.pth 。\

doremi_params_50000.pth 测试结果：\
imdb loss: 1.6527431011199951 \
imdb q-error: 30%: 1.5198   50%: 2.0029   80%: 4.1047   90%: 7.6879   95%: 19.0663   99%: 54.2403 \
model's p-error 30% 50% 80% 90% 95% 99%: [1.0, 1.0, 1.0537, 1.0939, 1.2081, 1.4636] \
stats loss: 4.141575336456299 \
stats q-error: 30%: 1.6441   50%: 2.494   80%: 9.3853   90%: 24.0765   95%: 62.5588   99%: 1307.7956 \
model's p-error 30% 50% 80% 90% 95% 99%: [1.0, 1.1762, 2.7326, 6.8562, 25.4018, 1288.4794] \
ergastf1 loss: 2.665390729904175 \
ergastf1 q-error: 30%: 1.8932   50%: 2.9233   80%: 7.6348   90%: 15.0011   95%: 25.0362   99%: 75.0406 \
model's p-error 30% 50% 80% 90% 95% 99%: [1.0, 1.0, 1.4272, 1.6641, 2.2407, 3.6265]
genome loss: 1.7334747314453125 \
genome q-error: 30%: 1.5125   50%: 1.8303   80%: 4.6351   90%: 11.2894   95%: 15.8277   99%: 85.4872
model's p-error 30% 50% 80% 90% 95% 99%: [1.0, 1.0, 1.0907, 1.29, 1.8453, 2.6369]

#### 初步分析4：
和 model_params.pth 的性能相比，在 imdb, genome 有明显提升，stats, ergastf1 下降了，但和已有的学习型方法比具有优势。和原始 price 模型相比，训练开销显著减少，原本需要50000*26条sql的训练集，现在只需要50015条sql。\
可以认为按照 avg_domain_weights_50000.json 中给出的比例继续增加训练集，可以进一步提升模型性能。现在使用50015条sql作为训练集的原因是 price 没有给出生成训练集的脚本，在 avg_domain_weights_50000.json 中占比最大的数据集是 \
baseball ，会使用49065条sql，因此没有用更多的sql。 \
根据 doremi 给出算法流程，实际应该先训练一个预训练模型 model_params_2000.pth ，即每个数据集采样2001条sql作为训练集，然后用 model_params_2000.pth 作为 reference model，训练 main model doremi_pretrain_params_2000.pth，\
得到的reweight保存为 avg_domain_weights_2000.json ，再用这组权重训练 doremi_params_2000.pth 并测试。这一条件下，训练集包含52038条sql，规模和50000条sql左右组成的训练集更接近，更方便分析 doremi 方案的优劣。 \

model_params_2000.pth 训练时输出：\
------------------------------------------------------------ \
epoch: 148, train loss: 4.418037120947553 \
train q-error: 30%: 1.9558   50%: 3.3112   80%: 11.682   90%: 29.38   95%: 72.2549   99%: 683.7058 \
------------------------------------------------------------ \
epoch: 149, train loss: 4.401393326216126 \
train q-error: 30%: 1.9526   50%: 3.282   80%: 11.6873   90%: 29.3735   95%: 72.2734   99%: 660.2382 \
done! 

model_params_2000.pth 测试结果：\
imdb loss: 1.8445366621017456 \
imdb q-error: 30%: 1.4221   50%: 1.8226   80%: 5.1022   90%: 8.2358   95%: 14.666   99%: 152.1815 \
stats loss: 3.206942319869995 \
stats q-error: 30%: 1.4814   50%: 2.0903   80%: 6.1004   90%: 15.4093   95%: 42.2334   99%: 735.1835 \
ergastf1 loss: 2.5535202026367188 \
ergastf1 q-error: 30%: 1.5378   50%: 2.1862   80%: 6.7038   90%: 14.9063   95%: 30.7945   99%: 134.1093 \
genome loss: 3.316244602203369 \
genome q-error: 30%: 1.1037   50%: 1.2856   80%: 3.9279   90%: 8.3571   95%: 121.3882   99%: 2183.6191 

doremi_params_2000.pth 训练时输出：\
------------------------------------------------------------ \
epoch: 148, train loss: 5.901407792056073 \
train q-error: 30%: 2.3126   50%: 4.4115   80%: 19.5908   90%: 54.3776   95%: 136.7614   99%: 1033.8256 \
------------------------------------------------------------ \
epoch: 149, train loss: 5.905209433231114 \
train q-error: 30%: 2.3039   50%: 4.3984   80%: 19.4681   90%: 53.4164   95%: 135.8139   99%: 1096.4772 \
done!

doremi_params_2000.pth 测试结果：\
imdb loss: 1.3971617221832275 \
imdb q-error: 30%: 1.3947   50%: 1.8509   80%: 3.263   90%: 5.9507   95%: 13.4404   99%: 71.9804 \
stats loss: 3.581186294555664 \
stats q-error: 30%: 1.9464   50%: 2.9404   80%: 8.0906   90%: 16.6173   95%: 48.8757   99%: 607.0037 \
ergastf1 loss: 3.22513747215271 \
ergastf1 q-error: 30%: 1.6982   50%: 2.5907   80%: 8.4776   90%: 19.9857   95%: 43.7435   99%: 202.4127 \
genome loss: 4.354820728302002 \
genome q-error: 30%: 1.2417   50%: 1.3386   80%: 6.0791   90%: 22.3583   95%: 249.8729   99%: 3332.5126 

#### 初步分析5：
model_params_2000.pth 和 论文中的 model_params.pth (每个数据集集上有50000条sql)相比，性能还是下降了很多。和 model_params_2000.pth 对应，doremi_params_2000.pth 方法在 imdb 上效果的提升显著，stats有所波动，\
ergastf1 和 genome 上反而有所下滑。可能需要尝试 Iterated DoReMi 方法。model_params_2000.pth 的训练集反而要比 doremi_params_50000.pth 的更多。

model_params_10000.pth 训练时输出：\
------------------------------------------------------------ \
epoch: 148, train loss: 1.3375686614728364 \
train q-error: 30%: 1.3223   50%: 1.6709   80%: 3.2751   90%: 6.0269   95%: 11.7419   99%: 60.0866 \
------------------------------------------------------------ \
epoch: 149, train loss: 1.3385464612577782 \
train q-error: 30%: 1.3223   50%: 1.6744   80%: 3.2795   90%: 6.022   95%: 11.6911   99%: 60.3108 \
done!

model_params_10000.pth 测试结果：\
imdb loss: 1.578393578529358  \
imdb q-error: 30%: 1.2838   50%: 1.8278   80%: 4.3873   90%: 8.7873   95%: 16.0471   99%: 71.3562 \
stats loss: 4.549349308013916 \
stats q-error: 30%: 1.3758   50%: 2.1437   80%: 8.5868   90%: 25.9533   95%: 112.1842   99%: 2965.5481 \
gergastf1 loss: 1.5752155780792236 \
ergastf1 q-error: 30%: 1.3653   50%: 1.8473   80%: 4.4014   90%: 7.8604   95%: 14.4038   99%: 53.9118 \
genome loss: 1.6679435968399048 \
genome q-error: 30%: 1.2719   50%: 1.6778   80%: 4.1561   90%: 7.2614   95%: 20.1704   99%: 89.9663 \

doremi_params_10000.pth 训练时输出：\
------------------------------------------------------------ \
epoch: 148, train loss: 6.168627891245837 \
train q-error: 30%: 2.36   50%: 4.5931   80%: 21.0066   90%: 57.8555   95%: 150.0806   99%: 1245.111 \
------------------------------------------------------------ \
epoch: 149, train loss: 6.094707413526826 \
train q-error: 30%: 2.3494   50%: 4.5683   80%: 20.9328   90%: 56.7961   95%: 144.2875   99%: 1145.9692 \
done!

doremi_params_10000.pth 测试结果：\
imdb loss: 2.2525687217712402 \
imdb q-error: 30%: 1.4948   50%: 1.8721   80%: 4.8979   90%: 12.4442   95%: 32.7431   99%: 187.8952 \
stats loss: 3.04954195022583 \
stats q-error: 30%: 1.8607   50%: 2.8275   80%: 6.8602   90%: 14.3384   95%: 38.9273   99%: 291.1859 \
ergastf1 loss: 2.9606130123138428 \
ergastf1 q-error: 30%: 1.5268   50%: 2.2283   80%: 8.214   90%: 17.2678   95%: 42.6139   99%: 182.5057 \
genome loss: 4.397848606109619 \
genome q-error: 30%: 1.2273   50%: 1.5071   80%: 5.456   90%: 21.9298   95%: 253.5204   99%: 3542.0721 \

#### 初步分析6：
出现了性能波动。但分析权重文件可以看出，随着参考模型使用训练集sql数量的增加，baseball 领域对应的权重在增加。

model_params_30000.pth 训练时输出：\
------------------------------------------------------------ \
poch: 148, train loss: 0.9905731532436151 \
train q-error: 30%: 1.2676   50%: 1.5438   80%: 2.694   90%: 4.4802   95%: 7.9779   99%: 36.6371 \
------------------------------------------------------------ \
epoch: 149, train loss: 0.9883709446741984 \
train q-error: 30%: 1.2675   50%: 1.5427   80%: 2.6866   90%: 4.4714   95%: 8.0052   99%: 36.536 \
done!

model_params_30000.pth 测试结果：\
imdb loss: 2.0140769481658936 \
imdb q-error: 30%: 1.3002   50%: 1.7316   80%: 4.539   90%: 8.708   95%: 17.2005   99%: 207.8552 \
input file for testing p-error saved in path: /home/user/oblab/PRICE/results/imdb_perror_input.sql \
stats loss: 7.029318809509277 \
stats q-error: 30%: 1.5757   50%: 2.8559   80%: 20.8411   90%: 74.9921   95%: 283.0125   99%: 8600.2667 \
ergastf1 loss: 1.4772710800170898 \
ergastf1 q-error: 30%: 1.2106   50%: 1.5756   80%: 3.7127   90%: 7.3212   95%: 18.1746   99%: 56.6376 \
genome loss: 4.023987293243408 \
genome q-error: 30%: 1.1355   50%: 1.4341   80%: 8.433   90%: 34.9067   95%: 156.1646   99%: 934.0842

doremi_params_30000.pth 测试结果：\
imdb loss: 1.1515259742736816 \
imdb q-error: 30%: 1.4181   50%: 1.8438   80%: 3.1296   90%: 6.2189   95%: 10.301   99%: 25.0355 \
stats loss: 3.721428871154785 \
stats q-error: 30%: 1.6103   50%: 2.4753   80%: 6.9425   90%: 17.0956   95%: 51.3578   99%: 1398.6411 \
ergastf1 loss: 1.8857240676879883 \
ergastf1 q-error: 30%: 1.4478   50%: 2.0401   80%: 4.6503   90%: 8.892   95%: 17.0382   99%: 102.3651 \
genome loss: 2.5375921726226807 \
genome q-error: 30%: 1.4988   50%: 2.253   80%: 3.9255   90%: 9.9118   95%: 47.9277   99%: 307.844

#### 初步分析7：
baseball 的领域权重确实进一步提升了，并且 doremi_params_30000.pth 的性能也有所提升，进一步接近 doremi_params_50000.pth。\
下一步可以测试将 baseball 的领域权重提升到100%的效果，计为 model_params_baseball.pth 。

model_params_baseball.pth 训练时输出：\
------------------------------------------------------------ \
epoch: 148, train loss: 5.867870097160339 \
train q-error: 30%: 2.2216   50%: 4.16   80%: 18.287   90%: 51.9656   95%: 142.351   99%: 1349.3663 \
------------------------------------------------------------ \
epoch: 149, train loss: 5.72373363494873 \
train q-error: 30%: 2.207   50%: 4.1336   80%: 17.8291   90%: 48.9445   95%: 130.5381   99%: 1217.6931 \
done!

model_params_baseball.pth 测试结果：\
imdb loss: 1.224959135055542 \
imdb q-error: 30%: 1.3262   50%: 1.687   80%: 2.9607   90%: 6.0323   95%: 11.8687   99%: 45.7446 \
stats loss: 4.104957103729248 \
stats q-error: 30%: 1.5727   50%: 2.3618   80%: 8.5833   90%: 24.3999   95%: 67.1609   99%: 1452.1898 \
ergastf1 loss: 2.781514883041382 \
ergastf1 q-error: 30%: 1.826   50%: 2.7556   80%: 8.0296   90%: 16.3697   95%: 30.4492   99%: 93.6252 \
genome loss: 1.6268903017044067 \
genome q-error: 30%: 1.3112   50%: 1.7492   80%: 2.5915   90%: 4.2462   95%: 25.6884   99%: 177.0285

#### 初步分析7：
baseball 的领域权重提升到100%即只用 baseball 数据集训练 price 得到的模型 model_params_baseball.pth 性能和 price提供的预训练模型 model_params.pth 以及 \
doremi_params_50000.pth 的性能差不多。\
下一步测试完全不用 baseball 的领域知识，计为 model_params_baseball_anti.pth 。 \
预期结果：\
a. 若 model_params_baseball_anti.pth 性能大幅劣于 model_params.pth 说明真正起作用的是 baseball 数据集，若 model_params_baseball_anti.pth 性能持平 model_params.pth 说明 baseball 数据集起到的作用等同于这25个数据集之和。 \
b. 若 model_params_baseball_anti.pth 性能优于 model_params_baseball_anti.pth 甚至 model_params.pth 的性能，则会否定前面的分析，需要重新设计实验。 \
当结果是 a 时，随机选取5个数据集单独训练 price 模型并测试性能。

model_params_baseball_anti.pth 训练时输出：\
------------------------------------------------------------ \
epoch: 148, train loss: 0.816954337644577 \
train q-error: 30%: 1.2397   50%: 1.4809   80%: 2.4353   90%: 3.8417   95%: 6.5567   99%: 27.1656 \
------------------------------------------------------------ \
epoch: 149, train loss: 0.8161604373693466 \
train q-error: 30%: 1.2398   50%: 1.4807   80%: 2.4321   90%: 3.838   95%: 6.5321   99%: 27.4532 \
done!

model_params_baseball_anti.pth 测试结果：\
imdb loss: 2.2040369510650635 \
imdb q-error: 30%: 1.3351   50%: 2.0995   80%: 4.9049   90%: 12.4819   95%: 24.608   99%: 179.2373 \
stats loss: 7.535816669464111 \
stats q-error: 30%: 1.6522   50%: 3.2201   80%: 26.9827   90%: 112.1595   95%: 401.6594   99%: 3252.4707 \
ergastf1 loss: 2.366539716720581 \
ergastf1 q-error: 30%: 1.198   50%: 1.5173   80%: 4.3304   90%: 12.4811   95%: 30.9691   99%: 275.8227 \
genome loss: 3.2539024353027344 \
genome q-error: 30%: 1.1523   50%: 1.2759   80%: 4.3152   90%: 18.4483   95%: 90.5534   99%: 1140.3316

#### 初步分析8：
model_params_baseball_anti.pth 的测试结果符合预期结果 a。因此将随机选取不是 baseball 的5个数据集作为测试。随机数 [24 17 19 25 12]，选取对应的单一数据集训练 price 模型，对应的数据集为 tournament，movielens，sap，tpc_h，fnhk， \
分别记为 model_params_tournament.pth，model_params_movielens.pth，model_params_sap.pth，model_params_tpc_h.pth，model_params_fnhk.pth。

model_params_tournament.pth 训练时输出：\
------------------------------------------------------------ \
epoch: 148, train loss: 3.848220343589783
train q-error: 30%: 1.9241   50%: 3.1739   80%: 10.4401   90%: 23.1533   95%: 50.4888   99%: 337.1064 \
------------------------------------------------------------ \
epoch: 149, train loss: 3.828664293289185 \
train q-error: 30%: 1.9137   50%: 3.1771   80%: 10.3778   90%: 23.4821   95%: 50.5343   99%: 347.9747 \
done!

model_params_tournament.pth 测试结果：\
imdb loss: 7.02153205871582 \
imdb q-error: 30%: 4.4638   50%: 7.0527   80%: 33.7032   90%: 70.6593   95%: 108.8551   99%: 333.0655 \
stats loss: 8.370213508605957 \
stats q-error: 30%: 2.6044   50%: 4.6724   80%: 34.4544   90%: 116.4985   95%: 411.7664   99%: 4960.4912 \
ergastf1 loss: 1.6940361261367798 \
ergastf1 q-error: 30%: 1.4462   50%: 1.9176   80%: 4.4772   90%: 8.789   95%: 17.4991   99%: 59.0462 \
genome loss: 3.3637373447418213 \
genome q-error: 30%: 1.648   50%: 2.3229   80%: 7.7152   90%: 18.9618   95%: 44.8754   99%: 589.9164 

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ \

model_params_movielens.pth 训练时输出：\
------------------------------------------------------------ \
epoch: 148, train loss: 4.7557932472229005
train q-error: 30%: 2.133   50%: 3.8269   80%: 14.6198   90%: 35.2873   95%: 81.6771   99%: 504.1262  \
------------------------------------------------------------ \
epoch: 149, train loss: 4.713628416061401 \
train q-error: 30%: 2.1167   50%: 3.8162   80%: 14.5058   90%: 35.4776   95%: 80.8229   99%: 482.5774 \
done!

model_params_movielens.pth 测试结果：\
imdb loss: 1.1206134557724
imdb q-error: 30%: 1.5121   50%: 1.8184   80%: 2.9578   90%: 4.5127   95%: 8.7858   99%: 41.4143
stats loss: 3.569838523864746
stats q-error: 30%: 2.0507   50%: 2.9924   80%: 7.7542   90%: 18.4462   95%: 45.3938   99%: 438.403
ergastf1 loss: 6.076362609863281
ergastf1 q-error: 30%: 1.8652   50%: 3.7098   80%: 21.6911   90%: 66.0699   95%: 161.5729   99%: 1279.2001
genome loss: 3.818009614944458
genome q-error: 30%: 1.1316   50%: 1.4159   80%: 4.4813   90%: 12.0365   95%: 178.9654   99%: 3109.3005

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ \

model_params_sap.pth 训练时输出：\
------------------------------------------------------------ \
epoch: 148, train loss: 3.5507129311561583
train q-error: 30%: 1.9347   50%: 3.2272   80%: 9.9627   90%: 21.4792   95%: 44.0588   99%: 216.8102 \
------------------------------------------------------------ \
epoch: 149, train loss: 3.493470063209534 \
train q-error: 30%: 1.9127   50%: 3.1681   80%: 9.748   90%: 20.7302   95%: 42.8445   99%: 219.1173 \
done!

model_params_sap.pth 测试结果：\
imdb loss: 1.6559627056121826 \
imdb q-error: 30%: 1.4765   50%: 1.981   80%: 4.633   90%: 8.8551   95%: 15.622   99%: 56.3414 \
stats loss: 3.6280691623687744 \
stats q-error: 30%: 1.565   50%: 2.5453   80%: 7.4729   90%: 17.525   95%: 72.475   99%: 610.2851 \
ergastf1 loss: 9.42021369934082 \
ergastf1 q-error: 30%: 4.0893   50%: 8.937   80%: 57.8911   90%: 163.6909   95%: 273.6026   99%: 1584.5554 \
genome loss: 7.331011772155762 \
genome q-error: 30%: 2.7103   50%: 5.5362   80%: 15.003   90%: 63.1612   95%: 434.9512   99%: 4633.6393

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ \

model_params_tpc_h.pth 训练时输出：\
------------------------------------------------------------ \
epoch: 148, train loss: 4.676386895179749 \
train q-error: 30%: 2.076   50%: 3.6771   80%: 13.8452   90%: 35.3026   95%: 83.6465   99%: 491.6702 \
------------------------------------------------------------ \
epoch: 149, train loss: 4.637356948852539 \
train q-error: 30%: 2.0723   50%: 3.6972   80%: 14.0759   90%: 34.6758   95%: 78.9653   99%: 472.4455 \
done!

model_params_tpc_h.pth 测试结果：\
imdb loss: 1.9065062999725342 \
imdb q-error: 30%: 1.2945   50%: 1.6707   80%: 3.4246   90%: 9.1804   95%: 19.4116   99%: 276.3743 \
stats loss: 3.3778369426727295 \
stats q-error: 30%: 1.5602   50%: 2.1198   80%: 5.5142   90%: 16.97   95%: 49.84   99%: 1354.4757 \
ergastf1 loss: 2.6508800983428955 \
ergastf1 q-error: 30%: 1.4411   50%: 2.0934   80%: 6.7875   90%: 16.9787   95%: 34.725   99%: 142.5555 \
genome loss: 3.369147539138794 \
genome q-error: 30%: 1.179   50%: 1.3601   80%: 2.3945   90%: 7.54   95%: 107.7843   99%: 5172.2996

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ \

model_params_fnhk.pth 训练时输出：\
------------------------------------------------------------ \
epoch: 148, train loss: 4.4587535572052 \
train q-error: 30%: 2.1086   50%: 3.726   80%: 13.2201   90%: 31.2281   95%: 68.3925   99%: 424.6123 \
------------------------------------------------------------ \
epoch: 149, train loss: 4.4292427110672 \
train q-error: 30%: 2.1041   50%: 3.7145   80%: 13.4401   90%: 31.2643   95%: 67.2848   99%: 376.0482 \
done!

model_params_fnhk.pth 测试结果：\
imdb loss: 12.042457580566406 \
imdb q-error: 30%: 2.4578   50%: 8.1605   80%: 55.8117   90%: 415.123   95%: 1216.4822   99%: 12986.5681 \
stats loss: 13.878193855285645 \
stats q-error: 30%: 2.1567   50%: 5.2028   80%: 98.034   90%: 673.3179   95%: 3884.2253   99%: 68668.8371 \
ergastf1 loss: 3.5786027908325195 \
ergastf1 q-error: 30%: 2.0305   50%: 3.6507   80%: 9.6648   90%: 21.6613   95%: 42.4154   99%: 259.9383 \
genome loss: 8.224937438964844 \
genome q-error: 30%: 1.9695   50%: 3.0626   80%: 32.5872   90%: 204.5753   95%: 423.1843   99%: 3214.8104 

#### 初步分析9：
可以看出随机选取的这5个数据集训练得到的 price 模型和 model_params_baseball.pth 的性能相比要差的多，符合 初步分析7、8的猜想。 \
在这26个训练集中，Hockey 的规模可能是最接近 baseball 的(# of Tables: 18 < 25, # of Join Relations: 145 > 103)，还应当测试下仅用 Hockey 数据集训练的模型 model_params_hockey.pth 性能。 \
如果性能接近 model_params_baseball.pth 的性能，可以进一步证实猜想；如果性能较差，说明 baseball 数据集存在某种特殊性，还需要具体分析。

model_params_hockey.pth 训练时输出：\
------------------------------------------------------------ \
epoch: 148, train loss: 2.576230330467224 \
train q-error: 30%: 1.6986   50%: 2.5737   80%: 6.9325   90%: 13.8166   95%: 26.7544   99%: 112.2668 \
------------------------------------------------------------ \
epoch: 149, train loss: 2.550199131965637 \
train q-error: 30%: 1.7014   50%: 2.5691   80%: 6.8954   90%: 13.5871   95%: 25.987   99%: 110.0643 \
done!

model_params_hockey.pth 测试结果：\
imdb loss: 1.3156168460845947 \
imdb q-error: 30%: 1.3476   50%: 1.7863   80%: 3.1132   90%: 4.9104   95%: 11.6366   99%: 70.6136 \
stats loss: 5.844807147979736 \
stats q-error: 30%: 1.7757   50%: 3.5847   80%: 18.9304   90%: 37.8191   95%: 140.7435   99%: 1728.0438 \
ergastf1 loss: 8.425570487976074 \
ergastf1 q-error: 30%: 4.9768   50%: 10.5899   80%: 41.9609   90%: 111.8768   95%: 157.2496   99%: 824.9336 \
genome loss: 3.447962522506714 \
genome q-error: 30%: 2.6127   50%: 3.413   80%: 8.0324   90%: 11.5405   95%: 51.6877   99%: 270.8551

#### 初步分析10：
当前时间2024/10/31，从初步分析9以来出现一篇可以作为参考的论文 cardbench ，非采样数据集和 price 重合，采样数据集的采样后形成的原始数据已向作者申请，目前没有回应。\
现在可以考虑测试 model_params_baseball.pth 在所有非 baseball 数据集上的性能，以及在 cardbench 中的模型使用 baseball 数据集训练的性能。\
cardbench 模型的相关内容在 cardbench 的路径下。\
model_params_baseball.pth 在所有非 baseball 数据集上每个数据集测试3000条数据。

model_params_baseball.pth 测试结果：\
accidents loss: 7.428691387176514 \
accidents q-error: 30%: 5.0093   50%: 10.2446   80%: 37.6447   90%: 61.4146   95%: 89.2242   99%: 194.9438 \
airline loss: 4.375038146972656 \
airline q-error: 30%: 2.7345   50%: 4.8689   80%: 13.786   90%: 23.8025   95%: 39.5438   99%: 241.4268 \
basketball loss: 4.759363651275635 \
basketball q-error: 30%: 2.3494   50%: 4.5353   80%: 16.1892   90%: 34.3328   95%: 67.1156   99%: 241.4814 \
carcinogenesis loss: 2.200101613998413 \
carcinogenesis q-error: 30%: 1.6524   50%: 2.8339   80%: 7.0375   90%: 11.0599   95%: 16.3495   99%: 44.4729 \
ccs loss: 3.2501986026763916 \
ccs q-error: 30%: 2.0194   50%: 3.498   80%: 10.5889   90%: 20.2261   95%: 31.8735   99%: 78.3201 \
chembl loss: 6.235239505767822 \
chembl q-error: 30%: 2.1849   50%: 4.1146   80%: 21.0196   90%: 64.5611   95%: 169.4411   99%: 1599.935 \
consumer loss: 0.7359192371368408 \
consumer q-error: 30%: 1.2361   50%: 1.5453   80%: 2.29   90%: 3.2471   95%: 6.4782   99%: 27.494 \
credit loss: 1.4549756050109863 \
credit q-error: 30%: 1.4548   50%: 2.0433   80%: 4.4117   90%: 7.3866   95%: 11.7947   99%: 31.0254 \
employee loss: 2.1578245162963867 \
employee q-error: 30%: 1.6814   50%: 2.5168   80%: 4.9216   90%: 12.4914   95%: 21.3111   99%: 105.1012 \
financial loss: 1.476922869682312 \
financial q-error: 30%: 1.4817   50%: 2.0737   80%: 4.6245   90%: 7.5701   95%: 11.7297   99%: 29.4087 \
fnhk loss: 6.36812686920166 \
fnhk q-error: 30%: 3.2765   50%: 6.447   80%: 23.1023   90%: 40.6745   95%: 76.7267   99%: 3063.3318 \
grants loss: 8.232094764709473 \
grants q-error: 30%: 2.6176   50%: 5.6753   80%: 33.0401   90%: 100.3947   95%: 292.5359   99%: 4260.1438 \
hepatitis loss: 0.6509532332420349 \
hepatitis q-error: 30%: 1.349   50%: 1.6995   80%: 2.7032   90%: 3.5895   95%: 4.942   99%: 10.4661 \
hockey loss: 1.956873893737793 \
hockey q-error: 30%: 1.4648   50%: 2.0425   80%: 5.2729   90%: 10.843   95%: 18.3893   99%: 70.8856 \
legalacts loss: 6.8510050773620605 \
legalacts q-error: 30%: 3.2107   50%: 9.735   80%: 39.5298   90%: 58.7977   95%: 69.0076   99%: 77.2119 \
movielens loss: 4.380982875823975 \
movielens q-error: 30%: 1.6718   50%: 2.5817   80%: 9.6659   90%: 34.2951   95%: 116.3914   99%: 1129.5233 \
sakila loss: 1.8918145895004272 \
sakila q-error: 30%: 1.3828   50%: 1.888   80%: 4.8058   90%: 11.0817   95%: 21.6572   99%: 60.4305 \
sap loss: 2.865196466445923 \
sap q-error: 30%: 1.9461   50%: 3.3087   80%: 9.1828   90%: 15.374   95%: 23.9076   99%: 66.2785 \
seznam loss: 1.8076331615447998 \
seznam q-error: 30%: 1.4933   50%: 1.9714   80%: 3.8047   90%: 7.6883   95%: 17.9276   99%: 118.095 \
ssb loss: 1.404941439628601 \
ssb q-error: 30%: 1.7679   50%: 2.3909   80%: 4.1418   90%: 5.99   95%: 9.02   99%: 28.4965 \
talkingdata loss: 9.34384822845459 \
talkingdata q-error: 30%: 2.3482   50%: 5.164   80%: 45.4396   90%: 186.0113   95%: 637.7124   99%: 4634.7422 \
telstra loss: 1.418856143951416 \
telstra q-error: 30%: 1.3439   50%: 1.7428   80%: 4.1871   90%: 7.8553   95%: 12.9039   99%: 35.8391 \
tournament loss: 7.367973804473877 \
tournament q-error: 30%: 3.8811   50%: 8.0455   80%: 33.1856   90%: 79.0135   95%: 152.4459   99%: 438.2383 \
tpc_h loss: 3.480602502822876 \
tpc_h q-error: 30%: 1.7251   50%: 2.7189   80%: 9.5143   90%: 23.1848   95%: 51.9244   99%: 264.7651 \
tubepricing loss: 3.079120397567749 \
tubepricing q-error: 30%: 1.6726   50%: 2.9559   80%: 9.3784   90%: 18.248   95%: 30.8585   99%: 118.8182

#### 初步分析11：
共测试25个数据集，对标 cardbench 中的图表，记录P50的q-error和P95的q-error。 \
P50 : [10.2446, 4.8689, 4.5353, 2.8339, 3.498, 4.1146, 1.5453, 2.0433, 2.5168, 2.0737, 6.447, 5.6753, 1.6995, 2.0425, 9.735, 2.5817, 1.888, 3.3087, 1.9714, 2.3909, 5.164, 1.7428, 8.0455, 2.7189, 2.9559] \
p95 : [89.2242, 39.5438, 67.1156, 16.3495, 31.8735, 169.4411, 6.4782, 11.7947, 21.3111, 11.7297, 76.7267, 292.5359, 4.942, 18.3893, 69.0076, 116.3914, 21.6572, 23.9076, 17.9276, 9.02, 637.7124, 12.9039, 152.4459, 51.9244, 30.8585] \
P50 最大为10.2446，P95 最大为292.5359，都明显好于 cardbench 中的结果，可以否定 cardbench 关于 zore shot pretrain 模型的结论。

根据这25个数据集 join forms 分类，目前我们绘制了全部 *Chain* 和 *Star* 类型数据集的graph结构，其中有些数据集之间graph结构的关系: \
    1. *Chain* 类型数据集：Accidents = Consumer = FNHK \
    2. *Star* 类型数据集：SSB ⊆ Airline \
        SSB ⊆ SAP, 虽然 SAP 也是属于 *Star* 类型数据集的，但是它graph结构中的有环。\
        Seznam ⊆ Telstra ⊆ Carcinogenesis 这些数据集的各个表之间都是全连接的。
    
设计实验测试：\
    1. 训练 Accidents price, Consumer price, FNHK price, 测试在自身以外另外2个的测试workload上的表现。因为 SSB 数据集的结构和 *Chain* 类型数据集的结构完全不同，比较 SSB price 在这三个测试workload上的表现。因为 SSB 数据集包含表的数量比这3个数据集要多，测试Accidents price, Consumer price, FNHK price 在 SSB 的测试workload上的表现。 \
    2. 训练 Airline price, SAP price 在 SSB 的测试workload上的表现。 \
    3. 训练 Carcinogenesis price 在 Telstra, Seznam 的测试workload上的表现。

accidents_pretrain_params.pth 测试结果：\
*consumer* loss: 13.099579811096191 \
pg q-error: 30%: 5.7159   50%: 22.36   80%: 354.0   90%: 1813.8557   95%: 6611.5356   99%: 68678.0234 \
consumer q-error: 30%: 5.6683   50%: 10.0525   80%: 218.1794   90%: 301.7065   95%: 355.7989   99%: 863.2725 \
*fnhk* loss: 6.731076717376709 \
pg q-error: 30%: 13.1423   50%: 91.7676   80%: 4647.1251   90%: 38726.211   95%: 192906.985   99%: 2104429.9481 \
fnhk q-error: 30%: 2.9218   50%: 5.2628   80%: 27.5151   90%: 55.8791   95%: 136.8091   99%: 2679.2793 \
*ssb* loss: 37.33415605102539
pg q-error: 30%: 6.3953   50%: 25.7374   80%: 462.6194   90%: 2392.2367   95%: 9146.6107   99%: 84659.2145
ssb q-error: 30%: 7.9826   50%: 59.2351   80%: 7342.7697   90%: 130995.8297   95%: 287450.7251   99%: 469981.7179

consumer_pretrain_params.pth 测试结果：\
*accidents* loss: 10.284721374511719 \
pg q-error: 30%: 12.1052   50%: 78.0701   80%: 3445.2859   90%: 29628.8945   95%: 175945.2493   99%: 3440839.636 \
accidents q-error: 30%: 5.869   50%: 12.7427   80%: 71.2161   90%: 149.4854   95%: 242.1053   99%: 887.3912 \
*fnhk* loss: 9.470232009887695 \
pg q-error: 30%: 13.1423   50%: 91.7676   80%: 4647.1251   90%: 38726.211   95%: 192906.985   99%: 2104429.9481 \
fnhk q-error: 30%: 2.7445   50%: 5.3302   80%: 23.809   90%: 152.08   95%: 469.8242   99%: 61580.1071 \
*ssb* loss: 6.056367191162109
pg q-error: 30%: 6.3953   50%: 25.7374   80%: 462.6194   90%: 2392.2367   95%: 9146.6107   99%: 84659.2145
ssb q-error: 30%: 2.7462   50%: 4.6163   80%: 15.0961   90%: 161.1869   95%: 311.8169   99%: 937.8744

fnhk_pretrain_params.pth 测试结果：\
*accidents* loss: 3.0136337280273438 \
pg q-error: 30%: 12.1052   50%: 78.0701   80%: 3445.2859   90%: 29628.8945   95%: 175945.2493   99%: 3440839.636 \
accidents q-error: 30%: 2.1262   50%: 2.9186   80%: 5.8857   90%: 11.1429   95%: 29.8727   99%: 481.248 \
*consumer* loss: 2.1140530109405518 \
pg q-error: 30%: 5.7159   50%: 22.36   80%: 354.0   90%: 1813.8557   95%: 6611.5356   99%: 68678.0234 \
consumer q-error: 30%: 1.2863   50%: 1.7965   80%: 7.3441   90%: 9.4682   95%: 17.6059   99%: 62.3916 \
*ssb* loss: 12.564857933044433
pg q-error: 30%: 6.3953   50%: 25.7374   80%: 462.6194   90%: 2392.2367   95%: 9146.6107   99%: 84659.2145
ssb q-error: 30%: 2.0007   50%: 6.1832   80%: 125.5902   90%: 1750.8781   95%: 3126.0665   99%: 5790.0163

ssb_pretrain_params.pth 测试结果：\
*accidents* loss: 3.5218660831451416 \
pg q-error: 30%: 12.1052   50%: 78.0701   80%: 3445.2859   90%: 29628.8945   95%: 175945.2493   99%: 3440839.636 \
accidents q-error: 30%: 1.8733   50%: 3.4787   80%: 10.7056   90%: 17.3233   95%: 30.4504   99%: 272.939 \
*consumer* loss: 2.150080680847168 \
pg q-error: 30%: 5.7159   50%: 22.36   80%: 354.0   90%: 1813.8557   95%: 6611.5356   99%: 68678.0234 \
consumer q-error: 30%: 2.0798   50%: 2.6439   80%: 4.342   90%: 6.6199   95%: 13.1525   99%: 264.9015 \
*fnhk* loss: 4.28316068649292 \
pg q-error: 30%: 13.1423   50%: 91.7676   80%: 4647.1251   90%: 38726.211   95%: 192906.985   99%: 2104429.9481 \
fnhk q-error: 30%: 2.1039   50%: 3.3791   80%: 10.6695   90%: 31.2177   95%: 78.9326   99%: 485.6011 \

sap_pretrain_params.pth 测试结果：\
*accidents* loss: 6.036838531494141 \
pg q-error: 30%: 12.1052   50%: 78.0701   80%: 3445.2859   90%: 29628.8945   95%: 175945.2493   99%: 3440839.636 \
accidents q-error: 30%: 3.307   50%: 6.89   80%: 26.9743   90%: 49.8968   95%: 75.4581   99%: 189.206 \
*consumer* loss: 0.907612144947052 \
pg q-error: 30%: 5.7159   50%: 22.36   80%: 354.0   90%: 1813.8557   95%: 6611.5356   99%: 68678.0234 \
consumer q-error: 30%: 1.4913   50%: 1.9371   80%: 2.9145   90%: 3.9806   95%: 5.9802   99%: 21.167 \
*fnhk* loss: 4.867251396179199 \
pg q-error: 30%: 13.1423   50%: 91.7676   80%: 4647.1251   90%: 38726.211   95%: 192906.985   99%: 2104429.9481 \
fnhk q-error: 30%: 1.6499   50%: 3.1117   80%: 14.6234   90%: 31.9551   95%: 58.9172   99%: 2913.2593 \
*ssb* loss: 1.2694385564804078 \
pg q-error: 30%: 6.3953   50%: 25.7374   80%: 462.6194   90%: 2392.2367   95%: 9146.6107   99%: 84659.2145 \
ssb q-error: 30%: 1.6411   50%: 2.3322   80%: 4.2325   90%: 6.0363   95%: 7.6238   99%: 11.8711 

airline_pretrain_params.pth 测试结果：\
*accidents* loss: 3.1780362129211426 \
pg q-error: 30%: 12.1052   50%: 78.0701   80%: 3445.2859   90%: 29628.8945   95%: 175945.2493   99%: 3440839.636 \
accidents q-error: 30%: 2.041   50%: 2.9325   80%: 7.1772   90%: 13.9108   95%: 31.6139   99%: 449.7653 \
*consumer* loss: 2.351644277572632 \
pg q-error: 30%: 5.7159   50%: 22.36   80%: 354.0   90%: 1813.8557   95%: 6611.5356   99%: 68678.0234 \
consumer q-error: 30%: 2.3492   50%: 3.4966   80%: 5.5444   90%: 7.1304   95%: 13.4566   99%: 117.6704 \
*fnhk* loss: 4.141179084777832 \
pg q-error: 30%: 13.1423   50%: 91.7676   80%: 4647.1251   90%: 38726.211   95%: 192906.985   99%: 2104429.9481 \
fnhk q-error: 30%: 1.9404   50%: 3.3497   80%: 10.8678   90%: 23.0281   95%: 69.3342   99%: 662.4298 \
*ssb* loss: 1.4902680575561524 \
pg q-error: 30%: 6.3953   50%: 25.7374   80%: 462.6194   90%: 2392.2367   95%: 9146.6107   99%: 84659.2145 \
ssb q-error: 30%: 1.5743   50%: 2.1734   80%: 5.037   90%: 7.9454   95%: 11.7099   99%: 23.8208
*telstra* loss: 8.222514792785644
pg q-error: 30%: 4.8597   50%: 16.6899   80%: 196.2275   90%: 699.5998   95%: 1761.9997   99%: 7363.4971
telstra q-error: 30%: 7.2773   50%: 10.917   80%: 37.2944   90%: 70.3634   95%: 122.9828   99%: 394.9742

carcinogenesis_pretrain_params.pth 测试结果：\
*accidents* loss: 5.261667120819092
pg q-error: 30%: 12.1912   50%: 79.4692   80%: 3442.4321   90%: 29686.1471   95%: 174190.2789   99%: 2891819.4645
accidents q-error: 30%: 3.0784   50%: 6.2658   80%: 19.052   90%: 35.0544   95%: 47.5712   99%: 101.5891
*consumer* loss: 4.320451641464233
pg q-error: 30%: 5.5959   50%: 22.0294   80%: 371.0383   90%: 1994.545   95%: 7958.5237   99%: 80816.88
consumer q-error: 30%: 1.956   50%: 4.177   80%: 11.9365   90%: 27.1121   95%: 39.0203   99%: 189.3352
*fnhk* loss: 3.680223297805786
pg q-error: 30%: 13.2532   50%: 94.471   80%: 5063.5178   90%: 38715.3701   95%: 185826.5073   99%: 1988551.0618
fnhk q-error: 30%: 1.6677   50%: 2.6714   80%: 8.6735   90%: 25.4885   95%: 49.949   99%: 430.0012
*ssb* loss: 13.19835930053711
pg q-error: 30%: 6.3953   50%: 25.7374   80%: 462.6194   90%: 2392.2367   95%: 9146.6107   99%: 84659.2145
ssb q-error: 30%: 3.7601   50%: 12.3826   80%: 162.8049   90%: 570.091   95%: 1288.7884   99%: 4096.997
*telstra* loss: 11.357225262451172
pg q-error: 30%: 4.8597   50%: 16.6899   80%: 196.2275   90%: 699.5998   95%: 1761.9997   99%: 7363.4971
telstra q-error: 30%: 5.0803   50%: 13.7804   80%: 102.5971   90%: 216.3765   95%: 401.0152   99%: 1776.4277

telstra_pretrain_params.pth 测试结果：\
*accidents* loss: 10.10588140258789
pg q-error: 30%: 12.1912   50%: 79.4692   80%: 3442.4321   90%: 29686.1471   95%: 174190.2789   99%: 2891819.4645
accidents q-error: 30%: 3.5045   50%: 9.8878   80%: 78.2169   90%: 172.7242   95%: 334.3069   99%: 715.1542
*consumer* loss: 0.9037854443359375
pg q-error: 30%: 5.5959   50%: 22.0294   80%: 371.0383   90%: 1994.545   95%: 7958.5237   99%: 80816.88
consumer q-error: 30%: 1.4776   50%: 1.7615   80%: 2.6031   90%: 4.229   95%: 6.5609   99%: 15.985
*fnhk* loss: 7.377566613006592
pg q-error: 30%: 13.2532   50%: 94.471   80%: 5063.5178   90%: 38715.3701   95%: 185826.5073   99%: 1988551.0618
fnhk q-error: 30%: 2.3258   50%: 5.5351   80%: 25.4954   90%: 63.777   95%: 143.5537   99%: 4253.8177
*ssb* loss: 2.5075517108917236
pg q-error: 30%: 6.3953   50%: 25.7374   80%: 462.6194   90%: 2392.2367   95%: 9146.6107   99%: 84659.2145
ssb q-error: 30%: 1.9469   50%: 3.0748   80%: 7.2404   90%: 14.1676   95%: 25.0889   99%: 64.5375

#### 初步分析12：
1.Accidents price, Consumer price, FNHK price 和 SAP price 在 ssb 测试workload及 Accidents, Consumer, FNHK 测试workload上的表现，可以发现准确率和数据集类型无关，但和数据集中表的数量或连接的数量有关。 \
2.Airline 和 SAP 数据集虽然结构不同，但都是包含了 SSB 数据集的结构，但基于这两个数据集所训练的模型和 Accidents price, Consumer price, FNHK price 在 ssb 测试workload的表现相比，具有更高的性能，这种结果基本符合我们的1.分析。
3.不好评价。

epoch: 145, train loss: 6.136642960833251
train q-error: 30%: 2.2583   50%: 4.326   80%: 20.2738   90%: 58.0445   95%: 157.0835   99%: 1439.751
------------------------------------------------------------
epoch: 146, train loss: 6.178534241107009
train q-error: 30%: 2.266   50%: 4.3218   80%: 20.3787   90%: 59.1979   95%: 167.4912   99%: 1404.5835
------------------------------------------------------------
epoch: 147, train loss: 6.172453494756588
train q-error: 30%: 2.2507   50%: 4.2692   80%: 20.3428   90%: 59.621   95%: 162.9269   99%: 1457.6653
------------------------------------------------------------
epoch: 148, train loss: 6.13531728690394
train q-error: 30%: 2.2387   50%: 4.2854   80%: 19.9167   90%: 59.1606   95%: 163.8404   99%: 1391.533
------------------------------------------------------------
epoch: 149, train loss: 6.09004689122675
train q-error: 30%: 2.2544   50%: 4.2777   80%: 19.9844   90%: 58.2309   95%: 156.8335   99%: 1312.4057


## 2025/05/06 PRICE修复训练集中存在错误的真实基数后，重新训练使用doremi算法评估的结果
(price) user@user-R8424-G11:PRICE$ python evaluate.py
2025-05-15 20:58:51
Namespace(bin_size=40, table_dim=4, filter_dim=43, query_hidden_dim=512, final_hidden_dim=1024, output_dim=1, n_embd=256, n_layers=6, n_heads=8, dropout_rate=0.2, batch_size=15000, lr=2.85e-05, wd=5e-05, step_size=5, gamma=0.9, epochs=150, domain_config_path='./datas/baseline_domain_weights.json', output_dir='./results', reweight_eta=1.0, reweight_eps=0.0001)
using imdb path: /home/user/oblab/PRICE/utils/model/../../setup/features/test/imdb/features40.pkl
imdb loading done!!, len: 766
using stats path: /home/user/oblab/PRICE/utils/model/../../setup/features/test/stats/features40.pkl
stats loading done!!, len: 2749
using ergastf1 path: /home/user/oblab/PRICE/utils/model/../../setup/features/test/ergastf1/features40.pkl
ergastf1 loading done!!, len: 1422
using genome path: /home/user/oblab/PRICE/utils/model/../../setup/features/test/genome/features40.pkl
genome loading done!!, len: 2302
max_n_join_col is 9, max_n_fanout is 12, max_n_table is 7, max_n_filter_col is 10
max_n_feature is 38!
dataset padding done!!
n_features: 38!
load model from /home/user/oblab/PRICE/results/doremi_new_pretrain_params.pth
------------------------------------------------------------
imdb loss: 1.1657719612121582
imdb q-error: 30%: 1.3221   50%: 1.6579   80%: 3.3948   90%: 5.3248   95%: 8.4837   99%: 47.2758
group_limits Num_of_underest pct_of_underest Num_of_overest pct_of_overest q_error_30% q_error_50% q_error_80% q_error_90% q_error_95% q_error_99%
[1e0, 1e1) 0.00  0.00  7.00  1.00  5.06  5.93  8.17  8.47  8.56  8.63
[1e1, 1e2) 3.00  0.17  15.00  0.83  2.47  3.53  11.63  13.02  13.15  13.75
[1e2, 1e3) 10.00  0.67  5.00  0.33  2.30  2.65  4.57  5.74  6.32  6.46
[1e3, 1e4) 19.00  0.56  15.00  0.44  2.09  2.70  5.14  6.99  8.14  9.55
[1e4, 1e5) 22.00  0.43  29.00  0.57  1.33  1.81  3.84  13.96  23.50  42.49
[1e5, 1e6) 81.00  0.47  90.00  0.53  1.22  1.37  2.02  3.75  4.40  148.47
[1e6, 1e7) 104.00  0.45  126.00  0.55  1.21  1.56  2.60  3.88  5.32  7.37
[1e7, 1e8) 83.00  0.50  84.00  0.50  1.32  1.71  3.35  4.95  7.47  9.54
[1e8, 1e9) 50.00  0.75  17.00  0.25  1.70  1.90  4.84  14.31  21.92  22.10
[1e9, 1e10) 4.00  1.00  0.00  0.00  21.60  34.44  47.28  47.28  47.28  47.28
[1e10, 1e11) 2.00  1.00  0.00  0.00  1.97  1.97  1.97  1.97  1.97  1.97
len(tags): 766, len(model_pred_cards): 766
input file for testing p-error saved in path: /home/user/oblab/PRICE/results/imdb_perror_input.sql
stats loss: 3.482184886932373
stats q-error: 30%: 1.4921   50%: 2.1337   80%: 5.6806   90%: 14.0866   95%: 41.8693   99%: 1923.8892
group_limits Num_of_underest pct_of_underest Num_of_overest pct_of_overest q_error_30% q_error_50% q_error_80% q_error_90% q_error_95% q_error_99%
[1e0, 1e1) 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00
[1e1, 1e2) 0.00  0.00  7.00  1.00  5.63  31.53  25386.42  443393.87  752957.96  1000609.23
[1e2, 1e3) 27.00  0.40  40.00  0.60  1.65  2.99  114.14  595.85  2099.72  1111454.12
[1e3, 1e4) 83.00  0.49  88.00  0.51  1.71  2.63  15.53  200.20  3082.88  13834.58
[1e4, 1e5) 331.00  0.47  375.00  0.53  1.32  1.69  3.53  6.29  22.73  1726.45
[1e5, 1e6) 270.00  0.32  567.00  0.68  1.35  1.76  4.43  9.52  16.79  63.19
[1e6, 1e7) 101.00  0.21  378.00  0.79  1.79  3.18  7.36  19.82  41.10  149.08
[1e7, 1e8) 66.00  0.21  241.00  0.79  1.81  2.72  5.91  12.76  39.91  216.16
[1e8, 1e9) 40.00  0.35  73.00  0.65  2.11  4.02  12.05  26.93  58.66  351.86
[1e9, 1e10) 24.00  0.86  4.00  0.14  2.44  3.05  10.38  26.26  50.95  102.97
[1e10, 1e11) 31.00  0.91  3.00  0.09  3.93  6.44  10.28  31.06  68.42  2310.49
len(tags): 2749, len(model_pred_cards): 2749
input file for testing p-error saved in path: /home/user/oblab/PRICE/results/stats_perror_input.sql
ergastf1 loss: 1.358346700668335
ergastf1 q-error: 30%: 1.2875   50%: 1.5982   80%: 3.5952   90%: 7.3137   95%: 13.801   99%: 37.823
group_limits Num_of_underest pct_of_underest Num_of_overest pct_of_overest q_error_30% q_error_50% q_error_80% q_error_90% q_error_95% q_error_99%
[1e0, 1e1) 0.00  0.00  7.00  1.00  2.91  3.01  20.25  21.15  21.65  22.06
[1e1, 1e2) 5.00  0.20  20.00  0.80  1.58  2.41  5.82  9.86  12.60  15.01
[1e2, 1e3) 31.00  0.40  46.00  0.60  1.38  1.55  3.85  8.82  27.02  33.53
[1e3, 1e4) 85.00  0.50  85.00  0.50  1.28  1.77  4.07  6.25  9.84  26.87
[1e4, 1e5) 153.00  0.62  95.00  0.38  1.21  1.47  2.83  6.48  10.76  33.36
[1e5, 1e6) 214.00  0.62  133.00  0.38  1.24  1.50  3.68  9.92  18.10  43.14
[1e6, 1e7) 180.00  0.67  88.00  0.33  1.30  1.49  2.47  4.47  11.17  28.08
[1e7, 1e8) 100.00  0.68  47.00  0.32  1.40  1.95  5.16  6.98  10.63  31.66
[1e8, 1e9) 72.00  0.78  20.00  0.22  1.51  2.13  3.07  6.62  18.26  165.40
[1e9, 1e10) 22.00  0.71  9.00  0.29  1.30  1.70  7.79  11.49  12.83  14.33
[1e10, 1e11) 8.00  0.80  2.00  0.20  2.43  3.84  27.87  82.63  82.63  82.63
len(tags): 1422, len(model_pred_cards): 1422
input file for testing p-error saved in path: /home/user/oblab/PRICE/results/ergastf1_perror_input.sql
genome loss: 2.4879724979400635
genome q-error: 30%: 1.4074   50%: 2.173   80%: 6.3593   90%: 7.5092   95%: 30.0471   99%: 143.1136
group_limits Num_of_underest pct_of_underest Num_of_overest pct_of_overest q_error_30% q_error_50% q_error_80% q_error_90% q_error_95% q_error_99%
[1e0, 1e1) 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00
[1e1, 1e2) 2.00  1.00  0.00  0.00  1.28  1.29  1.32  1.33  1.34  1.34
[1e2, 1e3) 3.00  0.21  11.00  0.79  1.19  1.41  2.38  5.41  7.64  9.22
[1e3, 1e4) 118.00  0.91  11.00  0.09  2.07  6.53  28.53  64.57  95.41  493.44
[1e4, 1e5) 180.00  0.88  25.00  0.12  1.52  1.93  3.16  7.87  19.80  29.07
[1e5, 1e6) 276.00  0.95  16.00  0.05  2.00  3.99  55.77  97.66  136.89  2490.71
[1e6, 1e7) 197.00  0.32  425.00  0.68  1.24  1.54  4.09  4.47  4.47  154.75
[1e7, 1e8) 10.00  0.02  577.00  0.98  1.32  1.43  1.68  2.49  2.52  2.56
[1e8, 1e9) 0.00  0.00  451.00  1.00  6.13  6.36  7.20  7.30  7.37  7.51
len(tags): 2302, len(model_pred_cards): 2302
input file for testing p-error saved in path: /home/user/oblab/PRICE/results/genome_perror_input.sql
done!
2025-05-15 20:58:56

load model from /home/user/oblab/PRICE/results/baseball_pretrain_params_r2_wf.pth
------------------------------------------------------------
imdb loss: 1.824169397354126
imdb q-error: 30%: 1.5124   50%: 1.9144   80%: 4.1879   90%: 11.0189   95%: 17.0987   99%: 81.2166
group_limits Num_of_underest pct_of_underest Num_of_overest pct_of_overest q_error_30% q_error_50% q_error_80% q_error_90% q_error_95% q_error_99%
[1e0, 1e1) 0.00  0.00  7.00  1.00  44.94  62.11  126.80  148.98  156.11  161.80
[1e1, 1e2) 0.00  0.00  18.00  1.00  32.16  43.69  99.92  112.99  132.15  176.66
[1e2, 1e3) 0.00  0.00  15.00  1.00  2.98  3.76  8.98  12.90  21.32  34.79
[1e3, 1e4) 4.00  0.12  30.00  0.88  1.42  2.02  6.22  12.86  28.92  28.92
[1e4, 1e5) 10.00  0.20  41.00  0.80  1.60  2.17  4.11  5.98  8.04  9.48
[1e5, 1e6) 26.00  0.15  145.00  0.85  1.51  1.91  3.75  4.52  11.18  17.42
[1e6, 1e7) 42.00  0.18  188.00  0.82  1.55  1.88  4.07  10.30  15.32  17.74
[1e7, 1e8) 34.00  0.20  133.00  0.80  1.41  1.63  3.18  6.32  11.04  61.41
[1e8, 1e9) 12.00  0.18  55.00  0.82  1.38  2.03  3.53  5.45  6.11  36.17
[1e9, 1e10) 4.00  1.00  0.00  0.00  3.32  4.12  4.92  4.92  4.92  4.92
[1e10, 1e11) 0.00  0.00  2.00  1.00  3.22  3.22  3.22  3.22  3.22  3.22
len(tags): 766, len(model_pred_cards): 766
input file for testing p-error saved in path: /home/user/oblab/PRICE/results/imdb_perror_input.sql
stats loss: 5.4529500007629395
stats q-error: 30%: 1.7811   50%: 3.0993   80%: 14.583   90%: 39.1224   95%: 121.5248   99%: 2548.0982
group_limits Num_of_underest pct_of_underest Num_of_overest pct_of_overest q_error_30% q_error_50% q_error_80% q_error_90% q_error_95% q_error_99%
[1e0, 1e1) 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00
[1e1, 1e2) 0.00  0.00  7.00  1.00  3.98  4.92  8220.53  142838.84  242558.60  322334.42
[1e2, 1e3) 43.00  0.64  24.00  0.36  3.05  5.40  32.61  240.54  1188.77  854361.83
[1e3, 1e4) 118.00  0.69  53.00  0.31  3.02  5.71  34.37  218.07  2187.33  12273.91
[1e4, 1e5) 485.00  0.69  221.00  0.31  1.61  2.59  11.17  29.42  68.51  2601.36
[1e5, 1e6) 437.00  0.52  400.00  0.48  1.65  2.81  11.00  22.80  46.29  250.60
[1e6, 1e7) 137.00  0.29  342.00  0.71  1.77  3.11  15.14  37.87  97.91  528.46
[1e7, 1e8) 72.00  0.23  235.00  0.77  1.53  2.42  18.57  115.43  201.40  1339.13
[1e8, 1e9) 50.00  0.44  63.00  0.56  4.16  7.52  29.62  62.76  107.65  212.51
[1e9, 1e10) 23.00  0.82  5.00  0.18  2.98  7.97  29.11  45.37  75.97  213.05
[1e10, 1e11) 34.00  1.00  0.00  0.00  7.39  11.79  21.64  32.52  152.88  2598.81
len(tags): 2749, len(model_pred_cards): 2749
input file for testing p-error saved in path: /home/user/oblab/PRICE/results/stats_perror_input.sql
ergastf1 loss: 1.7190155982971191
ergastf1 q-error: 30%: 1.5053   50%: 2.1056   80%: 4.7057   90%: 9.7489   95%: 15.4646   99%: 45.9086
group_limits Num_of_underest pct_of_underest Num_of_overest pct_of_overest q_error_30% q_error_50% q_error_80% q_error_90% q_error_95% q_error_99%
[1e0, 1e1) 0.00  0.00  7.00  1.00  1.50  1.73  8.15  10.47  12.03  13.28
[1e1, 1e2) 12.00  0.48  13.00  0.52  1.62  2.43  3.70  7.27  11.43  30.95
[1e2, 1e3) 54.00  0.70  23.00  0.30  1.45  2.05  4.05  7.58  11.78  22.42
[1e3, 1e4) 110.00  0.65  60.00  0.35  1.39  2.15  4.55  9.45  13.91  43.17
[1e4, 1e5) 190.00  0.77  58.00  0.23  1.61  2.16  3.85  7.90  11.74  22.06
[1e5, 1e6) 226.00  0.65  121.00  0.35  1.48  1.76  5.01  11.36  23.30  45.32
[1e6, 1e7) 136.00  0.51  132.00  0.49  1.30  1.79  3.95  6.92  12.56  28.03
[1e7, 1e8) 84.00  0.57  63.00  0.43  1.74  2.60  6.86  12.69  19.45  46.66
[1e8, 1e9) 72.00  0.78  20.00  0.22  1.99  2.82  6.21  9.84  29.40  145.16
[1e9, 1e10) 24.00  0.77  7.00  0.23  1.48  1.56  4.44  11.10  11.11  15.07
[1e10, 1e11) 9.00  0.90  1.00  0.10  3.34  3.88  23.79  59.76  59.76  59.76
len(tags): 1422, len(model_pred_cards): 1422
input file for testing p-error saved in path: /home/user/oblab/PRICE/results/ergastf1_perror_input.sql
genome loss: 1.7227658033370972
genome q-error: 30%: 1.4549   50%: 2.0597   80%: 5.3884   90%: 11.1018   95%: 14.6112   99%: 49.7199
group_limits Num_of_underest pct_of_underest Num_of_overest pct_of_overest q_error_30% q_error_50% q_error_80% q_error_90% q_error_95% q_error_99%
[1e0, 1e1) 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00
[1e1, 1e2) 0.00  0.00  2.00  1.00  2.14  2.19  2.28  2.30  2.32  2.33
[1e2, 1e3) 8.00  0.57  6.00  0.43  1.18  1.40  1.99  4.89  6.97  8.31
[1e3, 1e4) 110.00  0.85  19.00  0.15  1.44  1.93  12.64  27.96  47.75  58.77
[1e4, 1e5) 166.00  0.81  39.00  0.19  1.50  1.95  3.05  4.47  9.91  19.97
[1e5, 1e6) 216.00  0.74  76.00  0.26  1.44  2.08  11.05  27.45  45.09  93.35
[1e6, 1e7) 145.00  0.23  477.00  0.77  1.24  1.72  2.81  2.81  3.11  27.04
[1e7, 1e8) 0.00  0.00  587.00  1.00  1.43  1.93  2.29  3.27  4.00  4.04
[1e8, 1e9) 0.00  0.00  451.00  1.00  5.38  6.21  14.44  14.60  14.63  35.57
len(tags): 2302, len(model_pred_cards): 2302
input file for testing p-error saved in path: /home/user/oblab/PRICE/results/genome_perror_input.sql
done!
2025-05-22 18:17:22

训练时间 2025-05-20 16:16:47 ~ 2025-05-21 20:13:12 price
训练时间 2025-05-15 19:08:23 ~ 2025-05-15 20:56:18 simple price
训练时间 2025-05-22 11:32:53 ~ 2025-05-22 13:08:15 baseball price (10W) n_features 41 训练集在/home/user/oblab/CE-baselines/test_dataset_training/workloads/baseball/features40.pkl